In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import random
import os
import time

from sklearn.metrics import f1_score,accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split

from model import BaseModel
from dataloader import BertDataset
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
params = {
    'EPOCHS':3,
    'LEARNING_RATE':2e-5,
    'BATCH_SIZE':16,
    'SEED':45,
    'DATA_PATH': '../../make_data/preprocessed_data/good_bad_df.csv',
    'SAVE_PATH':'../Models/Bert_Sentence.pt',
    'transformer':"bert-base-multilingual-cased",
    'max_length':512
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(params['SEED']) # Seed 고정

In [4]:
data = pd.read_csv(params['DATA_PATH'],index_col=[0])

data.loc[data['ad_label']==1,'ad_label']=0
data.loc[data['ad_label']==2,'ad_label']=1

train_data, test_data = train_test_split(data,test_size=0.2,random_state=params['SEED'],shuffle=True)
valid_data, test_data = train_test_split(test_data,test_size=0.5,random_state=params['SEED'],shuffle=True)

train_data = train_data.sample(frac=1).reset_index(drop=True)
valid_data = valid_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

train_data.reset_index(drop=True,inplace=True)
valid_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [5]:
print(len(train_data))
print(len(train_data.drop_duplicates(['summary'])))

print(len(valid_data))
print(len(valid_data.drop_duplicates(['summary'])))

print(len(test_data))
print(len(test_data.drop_duplicates(['summary'])))

print(train_data['ad_label'].value_counts()/len(train_data))
print(valid_data['ad_label'].value_counts()/len(valid_data))
print(test_data['ad_label'].value_counts()/len(test_data))

6867
6867
858
858
859
859
0    0.696665
1    0.303335
Name: ad_label, dtype: float64
0    0.715618
1    0.284382
Name: ad_label, dtype: float64
0    0.704307
1    0.295693
Name: ad_label, dtype: float64


In [6]:
train_dataset = BertDataset(train_data,params)
train_dataloader = DataLoader(train_dataset,batch_size=params['BATCH_SIZE'])
valid_dataset = BertDataset(valid_data,params)
valid_dataloader = DataLoader(valid_dataset,batch_size=params['BATCH_SIZE'])
test_dataset = BertDataset(test_data,params)
test_dataloader = DataLoader(test_dataset,batch_size=params['BATCH_SIZE'])

In [7]:
def train(model, train_loader,valid_loader, params, device):
    
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(),lr=params['LEARNING_RATE'])
    
    best_score = 0
    best_model = "None"
    for epoch_num in range(1,params["EPOCHS"]+1):
        
        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        timings=np.zeros((len(train_loader),1))
        cnt = 0
        
        model.train()
        
        train_loss = []
        for input_ids,masks,labels in tqdm(train_loader):
            
            starter.record()
            
            train_input_ids = input_ids.to(device)
            train_masks = masks.to(device)
            train_labels = labels.to(device)
            
            optimizer.zero_grad()

            output = model(train_input_ids,train_masks)     
            output = output.reshape(-1)
            
            batch_loss = criterion(output.to(torch.float32), train_labels.to(torch.float32)) 
            train_loss.append(batch_loss.item())
            
            batch_loss.backward()
            optimizer.step()
            
            ender.record()
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[cnt] = curr_time
            cnt += 1
        val_loss, val_acc, val_f1 = validation(model, criterion, valid_loader, device)
        
        time.sleep(1)
        print(f'Epoch [{epoch_num}], Train Loss : [{np.mean(train_loss) :.5f}] \
              Val Loss : [{np.mean(val_loss) :.5f}] Val Accuracy Score : [{val_acc:.5f}] Val F1 Score : [{val_f1:.5f}]')
        print('time per batch',np.mean(timings))
        
        val_score = val_f1
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model                         

In [8]:
def competition_metric(true, pred):
    return accuracy_score(true,pred),f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    val_loss = []
    model_preds = []
    true_labels = []  
    with torch.no_grad():
        for input_ids, masks, labels in tqdm(test_loader):
            valid_labels = labels.to(device)
            valid_input_ids = input_ids.to(device)
            valid_masks = masks.to(device)

            output = model(valid_input_ids, valid_masks)
            output = output.reshape(-1)
            
            batch_loss = criterion(output.to(torch.float32), valid_labels.to(torch.float32)) 
            val_loss.append(batch_loss.item())      
            
            output[output>0.5] = 1
            output[output<=0.5] = 0
            model_preds += output.detach().cpu().numpy().tolist()
            true_labels += valid_labels.detach().cpu().numpy().tolist()
        val_acc, val_f1 = competition_metric(true_labels, model_preds)
    return val_loss, val_acc, val_f1    

In [9]:
def inference(model, test_loader, device):
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    timings=np.zeros((len(test_loader),1))
    
    model.to(device)
    model.eval()
    
    test_predict = []
    true_labels = []
    
    with torch.no_grad():
        cnt = 0
        for input_ids, masks, labels in tqdm(test_loader):
            starter.record()
            
            test_labels = labels.to(device)
            test_input_ids= input_ids.to(device)
            test_masks = masks.to(device)

            output = model(test_input_ids, test_masks)  
            
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[cnt] = curr_time
            cnt += 1
            
            output[output>0.5] = 1
            output[output<=0.5] = 0
            test_predict += output.detach().cpu().numpy().tolist()
            true_labels += test_labels.detach().cpu().numpy().tolist()
            
    time.sleep(1)
    print('Done.')
    print('time per batch',np.mean(timings))
    return test_predict

In [10]:
model = BaseModel(params)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = params["LEARNING_RATE"])

infer_model = train(model, train_dataloader, valid_dataloader, params, device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 54/54 [00:06<00:00,  8.36it/s]


Epoch [1], Train Loss : [0.23186]               Val Loss : [0.14424] Val Accuracy Score : [0.95921] Val F1 Score : [0.94781]
time per batch 347.9664827568586


100%|██████████| 54/54 [00:06<00:00,  8.35it/s]


Epoch [2], Train Loss : [0.07184]               Val Loss : [0.06673] Val Accuracy Score : [0.97902] Val F1 Score : [0.97383]
time per batch 348.235297056686


100%|██████████| 54/54 [00:06<00:00,  8.33it/s]


Epoch [3], Train Loss : [0.04235]               Val Loss : [0.05960] Val Accuracy Score : [0.98601] Val F1 Score : [0.98278]
time per batch 348.3799065168514


In [16]:
test_labels = test_data['ad_label']
test_preds = inference(infer_model,test_dataloader,device)

print(competition_metric(test_labels,test_preds))

print(confusion_matrix(test_labels,test_preds))

100%|██████████| 54/54 [00:06<00:00,  8.50it/s]


Done.
time per batch 112.16902485600224
(0.9848661233993015, 0.9816417083551845)
[[603   2]
 [ 11 243]]


In [17]:
train_labels = train_data['ad_label']
train_preds = inference(infer_model,train_dataloader,device)

print(competition_metric(train_labels,train_preds))

print(confusion_matrix(train_labels,train_preds))

100%|██████████| 430/430 [00:51<00:00,  8.42it/s]


Done.
time per batch 113.3750128901282
(0.9975243920198049, 0.9970708985862085)
[[4776    8]
 [   9 2074]]


In [13]:
valid_labels = valid_data['ad_label']
valid_preds = inference(infer_model,valid_dataloader,device)

100%|██████████| 54/54 [00:06<00:00,  8.34it/s]


Done.
time per batch 114.57641417891891


In [14]:
print(competition_metric(valid_labels,valid_preds))

print(confusion_matrix(valid_labels,valid_preds))

(0.986013986013986, 0.9827762722071665)
[[609   5]
 [  7 237]]


In [15]:
torch.save(infer_model.state_dict(),params['SAVE_PATH'])

In [20]:
print("infermodel : ", sum(p.numel() for p in infer_model.parameters() if p.requires_grad))

infermodel :  177952001
